In [2]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from circuits.QAOA import QAOA_random
from src.GCP_hypergraph import QuantumCircuitHyperGraph
from src.FM_main import *
from src.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_QV_rec_5it.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_QV_rec_5it.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(64, 96, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 8 + i

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
        circuit = QuantumVolume(num_qubits,num_qubits)
        circuit = transpile(circuit, basis_gates = ['u','cp'])
        base_graph = QuantumCircuitHyperGraph(num_qubits,depth=circuit.depth())
        base_graph.map_circuit_to_hypergraph(circuit,group_gates=True)
        depth = base_graph.depth
        initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
        
        # -------------------------
        # 2. Fine-grained partitioning
        # -------------------------
        graph_list = [base_graph]
        mapping_list = [{i : set([i]) for i in range(depth)}]
        
        # -------------------------
        # 5. Recursive refinement
        # -------------------------
        assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                    initial_assignment,  
                                    qpu_info, 
                                    limit = None, 
                                    pass_list= None, 
                                    stochastic=True, 
                                    lock_nodes=False,
                                    log = False,
                                    add_initial = False,
                                    costs = None)
        
        total_time_r = sum(time_list_r)
        min_cost_r = min(cost_list_r)

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "r_cost":  min_cost_r,
            "time_r": total_time_r,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["r_cost"] for x in iteration_data]
    

    r_time_list = [x["time_r"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 64, # Partitions: 8
Mean Costs:
  Recursive:2124.200
Mean Times (s):
  Recursive:93.414
Finished 10 iterations for:
  # Qubits: 72, # Partitions: 9
Mean Costs:
  Recursive:3232.400
Mean Times (s):
  Recursive:149.105
Finished 10 iterations for:
  # Qubits: 80, # Partitions: 10
Mean Costs:
  Recursive:3830.800
Mean Times (s):
  Recursive:250.917
Finished 10 iterations for:
  # Qubits: 88, # Partitions: 11
Mean Costs:
  Recursive:4585.600
Mean Times (s):
  Recursive:317.365
Benchmarking completed. Detailed results saved to benchmark_results_FM_QV_rec_5it.json
Aggregated means saved to benchmark_means_FM_QV_rec_5it.json
